# 05 Python und Streamlit

Svelte war richtig cool! Vor allem, weil man sofort sehen kann, was man verändert hat - und es ziemlich einfach ist, Bedienknöpfe und Schieberegler einzubauen. Wenn man den Komfort doch auch nur in Python hätte... (Diese Notebooks, wie das hier, sind ja deutlich umständlicher, auch wenn sie einem die schönen ipywidgets aus [Übung 03](./03_interaktive_widgets.ipynb) zur Verfügung stellen.)

Das Schöne: es gibt eine Python-Erweiterung, mit der man an den Komfort von Svelte zumindest herankommt. [Streamlit](https://streamlit.io/) macht es einfach, Python-Apps mit Eingabe- und Ausgabe-Elementen zu bauen - und sich gleich anzuschauen, was man im Code verändert hat.  

## Streamlit einbauen und testen

Zuerst brauchen wir die ```streamlit```-Bibliothek (und das begleitende ```watchdog```-Paket, das die Ausführung etwas beschleunigt). 

Die nächste Codebox ausführen (oder die Zeile im Terminal eingeben, ohne das Ausführungszeichen): 

In [ ]:
!pip install streamlit watchdog

**So, aber wie benutzt man das jetzt?** Schauen wir uns das einfach Streamlit-Programm im nächsten Codeblock an: 

- Es druckt "Hello World!"
- Es zaubert einen Schieberegler auf den Schirm... 
- ...und zeigt das Ergebnis einer Berechnung, wenn wir den Regler verschieben. 

Allerdings: wenn wir die nächste Codebox in diesem Workbook ausführen, bekommen wir... nur Fehlermeldungen. 

Streamlit läuft nicht in Notebooks. Streamlit-Programme startet man auf der Kommandozeile mit

```
streamlit run <dateiname.py>
```

Dafür den Codeblock als Extra-Datei abspeichern: 

- Eine leere Python-Datei anlegen (im VSC-Menü über "Datei/Neue Datei..." oder das Neues-Dokument-Symbol im VSC-Datei-Explorer)
- Den Inhalt des Codeblocks unten in die leere Datei kopieren
- Abspeichern als ```streamlit_hello_world.py```

In [ ]:
# Die einfachst mögliche Streamlit-App

import streamlit as st
hello_str = "World"
st.write(f"Hello {hello_str}!")

# ...mit einer kleinen Erweiterung:
x = st.slider("Gib einen Wert mit dem Slider ein:",
              min_value = 0,
              max_value = 100,
              value = 1)
st.write(x, "zum Quadrat ist", x * x)

Die Datei können wir jetzt entweder über das Terminal ausführen - oder über einen Klick mit einer kleinen Erweiterung: 

## VSC-Erweiterung installieren

Über die Erweiterung [https://marketplace.visualstudio.com/items?itemName=joshrmosier.streamlit-runner](Streamlit-Runner) kann man Streamlit-Apps direkt in VSC ausführen: Rechtsklick auf die Datei, "Run with streamlit" auswählen - das öffnet ein Browserfenster, in dem die App ausgeführt wird. 

![Screenshot: Kontextmenü für Datei in VSC nach Rechtsklick](bilder/streamlit.png)

## Wenn ich etwas ändere...

...und die Änderung in VSC speichere, bekommt Streamlit das mit - und fragt mich am oberen rechten Browserfenster-Rand, ob es die Änderungen ausführen soll: 

![Screenshot: Kontextmenü für Datei in VSC nach Rechtsklick](bilder/streamlit_rerun.png)

Wer auf "Always rerun" klickt, bekommt jede Änderung angezeigt, sobald sie gespeichert ist. 

## Stoppen

Die Streamlit-Anwendung läuft solange, bis VSC beendet wird - oder das Programm im Terminal mit Ctrl-C bzw. Cmd-C beendet wird. 

# Aufgabe: 

Unten ist eine kleine Beispielanwendung für einen sehr simplen Claude-Chatbot. 

- Kannst du ihn zum Laufen bringen?
- Kannst du ihn so umbauen, dass er ```ollama``` und ein lokales KI-Modell wie das sehr genügsame ```llama3.2``` nutzt?


In [ ]:
# Basiert auf der Streamlit-Demo-Anwendung: 
# - https://github.com/streamlit/llm-examples
# 
# Dokumentation der Bibliotheken für Anthropic/Claude und Ollama: 
# - https://github.com/anthropics/anthropic-sdk-python
# - https://github.com/ollama/ollama-python

from anthropic import Anthropic
import streamlit as st


with st.sidebar:
    api_key = st.text_input("Anthropic API Key", key="chatbot_api_key", type="password")
    # Strings werden als Markdown interpretiert - das hier ist ein Link!
    "[API-Key bei Anthropic bekommen](https://console.anthropic.com/settings/keys)"
    

st.title("💬 Claude-Chatbot")
st.caption("🚀 Eine einfache Streamlit-Anwendung, die die Anthropic-KI nutzt")
if "messages" not in st.session_state:
    st.session_state["messages"] = [{"role": "assistant", "content": "Was kann ich für dich tun?"}]

for msg in st.session_state.messages:
    st.chat_message(msg["role"]).write(msg["content"])

if prompt := st.chat_input():
    if not api_key:
        st.info("Bitte erst einen Anthropic-API-Key eintragen.")
        st.stop()

    client = Anthropic(api_key=api_key)
    st.session_state.messages.append({"role": "user", "content": prompt})
    st.chat_message("user").write(prompt)
    response = client.messages.create(
        max_tokens = 1024,
        model="claude-3-5-sonnet-latest", 
        messages=st.session_state.messages)
    msg = response.content[0].text
    st.session_state.messages.append({"role": "assistant", "content": msg})
    st.chat_message("assistant").write(msg)